In [9]:
import json, pandas as pd
from sqlalchemy import Text, create_engine, Integer, String, Column, DateTime, Float, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import sqlalchemy
from sqlalchemy.types import TypeDecorator
import logging


SIZE = 256


class TextPickleType(TypeDecorator):

        """
    
        You can create a custom type by subclassing sqlalchemy.types.TypeDecorator to handle serialization and deserialization to Text:
        - https://stackoverflow.com/questions/1378325/python-dicts-in-sqlalchemy
        - http://docs.sqlalchemy.org/en/latest/core/custom_types.html
    
        """

        impl = sqlalchemy.Text(SIZE)

        def process_bind_param(self, value, dialect):
            if value is not None:
                value = json.dumps(value)

            return value

        def process_result_value(self, value, dialect):
            if value is not None:
                value = json.loads(value)
            return value

# echo controls how verbose sqlalchemy is - set to True to more information shown in the console
engine = create_engine('sqlite:///DataBase/sleep_performance.db', echo = False)

Base = declarative_base()


# we do a one to many relational database scheme

class Time(Base):
    __tablename__ 	= 'times'

    id 		= Column(Integer, primary_key = True)
    created_at 	= Column(DateTime, default = datetime.now(), onupdate = datetime.now())
    n_split		= Column(Integer)
    program_duration= Column(String)
    cov_type	= Column(String)
    label_type	= Column(String)
    
    subjects	= relationship("Subject", cascade="save-update")
    
    def __repr__(self):
        return "<Time(time_id = {5}, created_at= {0}, n_split= {1}, program_duration= {2}, cov_type={3}, label_type={4})>".format(
                                    self.created_at, self.n_split, self.program_duration, self.cov_type, self.label_type, self.id)
    
class Subject(Base):
    __tablename__	= 'subjects'
    
    id		= Column(Integer, primary_key = True)
    subject		= Column(String)
    class_prior 	= Column(TextPickleType())	
    total_sleep 	= Column(Float)
    nnmf_dim	= Column(Integer)
    nnmf_error	= Column(Float)
    time_id		= Column(Integer, ForeignKey('times.id'))
    
    # back_populates need to be set to the relationshi variable in the parent
    fracs		= relationship("Frac", cascade="save-update") 

    def __repr__(self):
        return "<Subject(subject_id = {6}, subject= {0}, class_prior= {1}, total_sleep= {2}, nnmf_dim={3}, nnmf_error={4}, time_id={5})>".format(
                                    self.subject, self.class_prior, self.total_sleep, self.nnmf_dim, self.nnmf_error, self.time_id, self.id)

class Frac(Base):
    __tablename__	= 'fracs'
    
    id		= Column(Integer, primary_key = True)
    frac		= Column(String)
    best_params 	= Column(TextPickleType())	
    best_val_score 	= Column(Float)	
    test_score	= Column(Float)
    subject_id	= Column(Integer, ForeignKey('subjects.id'))

    def __repr__(self):
        return "<Frac(frac_id = {5}, frac= {0}, best_params= {1}, best_val_score= {2}, test_score={3}, subject_id={4})>".format(
                                    self.frac, self.best_params, self.best_val_score, self.test_score, self.subject_id, self.id)
    
    
### Create_all does not recreate/modify already existing tables. Thus if new columes needs added you have to drop old tables
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


# other scores are f1 and others.... Implement


Session		= sessionmaker(bind = engine)
session 	= Session()


# function for parsing all values into strings of the dict in order for the sql database for properly handle the dict
def parse_dict_to_string(params):
    return {key: str(value) for key, value in params.iteritems()}

def extract_non_dict(a_dict, dict_name = None):
    new_dict 		= None
    
    if dict_name:
        dict_name, name_num 	= dict_name.split("_")
        extracted 		= {dict_name: dict_name + name_num}
    else:
        extracted 		= {}
    for key, value in a_dict.iteritems():
        
        if not isinstance(value,dict) :
            extracted.update({key:value})
        else:
            new_dict = value
    
    return extracted, new_dict

def to_sql(performance_dict):
    time_dict, run_results_dict = extract_non_dict(performance_dict)

    entry			= Time(**time_dict)

    for subject_key, sub_dict in run_results_dict.iteritems():
        extrat_subject_info, frac_dict 		= extract_non_dict(sub_dict, subject_key)
        sub = Subject(**extrat_subject_info)
        for frac_key, frac_dict in frac_dict.iteritems():
            extract_subject_info, frac_sql	= extract_non_dict(frac_dict, frac_key)
            sub.fracs.append(Frac(**extract_subject_info))
        
        entry.subjects.append(sub)
    
    session.add(entry)
    session.commit()

In [10]:
for col in Time.__table__.columns:
    print "col: " + col.description

dict1 = {'Name': 'Zara', 'Age': 7, 'Class': 'First'}
dict2 = {'Name': 'Sara', 'Age': 8, 'Class': 'Second'}
dict3 = {'Name': 'Cara', 'Age': 9, 'Class': 'Third'}
dict4 = {'Name': 'Lara', 'Age': 10, 'Class': 'Fourth'}
dict5 = {'Name': 'Zara', 'Age': 11, 'Class': 'First'}
dict6 = {'Name': 'Sara', 'Age': 12, 'Class': 'Second'}
testin = Time(n_split=1,program_duration='test_duration',cov_type='test_cov',label_type='test_label')
testin2 = Time(n_split=2,program_duration='test_duration2',cov_type='test_cov2',label_type='test_label2')
testsub1 = Subject(subject='test_subject1',class_prior=dict1,total_sleep=1.1,nnmf_dim=1,nnmf_error=1.1)
testsub2 = Subject(subject='test_subject2',class_prior=dict2,total_sleep=2.2,nnmf_dim=2,nnmf_error=2.2)
testsub3 = Subject(subject='test_subject2',class_prior=dict2,total_sleep=2.2,nnmf_dim=2,nnmf_error=2.2) #Same as previous
testsub4 = Subject(subject='test_subject3',class_prior=dict4,total_sleep=3.3,nnmf_dim=3,nnmf_error=3.3)
testfrac1 = Frac(frac='test_frac1',best_params=dict1,best_val_score=1.1,test_score=1.1)
testfrac2 = Frac(frac='test_frac2',best_params=dict2,best_val_score=2.2,test_score=2.2)
testfrac3 = Frac(frac='test_frac2',best_params=dict2,best_val_score=2.2,test_score=2.2)#Same as previous
testfrac4 = Frac(frac='test_frac3',best_params=dict3,best_val_score=3.3,test_score=3.3)
testfrac5 = Frac(frac='test_frac4',best_params=dict4,best_val_score=4.4,test_score=4.4)
testfrac6 = Frac(frac='test_frac5',best_params=dict5,best_val_score=5.5,test_score=5.5)
testfrac7 = Frac(frac='test_frac5',best_params=dict5,best_val_score=5.5,test_score=5.5)#Same as previous
testfrac8 = Frac(frac='test_frac6',best_params=dict6,best_val_score=6.6,test_score=6.6)

col: id
col: created_at
col: n_split
col: program_duration
col: cov_type
col: label_type


In [11]:
#No name may be called twice and everything must be intialized as above (so no copies testfrac7=testfrac6). Otherwise dublicate rows will be deleted
testsub1.fracs = [testfrac1, testfrac2]
testsub2.fracs = [testfrac3, testfrac4]
testsub3.fracs = [testfrac5, testfrac6]
testsub4.fracs = [testfrac7, testfrac8]
testin.subjects = [testsub1, testsub2]
testin2.subjects = [testsub3, testsub4]

session.add(testin)
session.add(testin2)
session.commit()

In [13]:
def print_sql_columns():
    print "Table Time contains columns:"
    for col in Time.__table__.columns:
        print "col: " + col.description
    print "Table Subject contains columns:"
    for col in Subject.__table__.columns:
        print "col: " + col.description
    print "Table Frac contains columns:"
    for col in Frac.__table__.columns:
        print "col: " + col.description
        
def print_run_times():
    for id, created_at in session.query(Time.id, Time.created_at):
        print (id, created_at)
        
def print_run_results(time_id):
    for t in session.query(Time).filter(Time.id==time_id).all():
        print t
        for s in session.query(Subject).filter(Subject.time_id==time_id).all(): 
            print s
            s_id = s.id
            for f in session.query(Frac).filter(Frac.subject_id==s.id).all(): 
                print f
    
    
#print_sql_columns()
#print_run_times()
print_run_results(2)

<Time(time_id = 2, created_at= 2017-08-31 18:34:25.660000, n_split= 2, program_duration= test_duration2, cov_type=test_cov2, label_type=test_label2)>
<Subject(subject_id = 3, subject= test_subject2, class_prior= {u'Age': 8, u'Name': u'Sara', u'Class': u'Second'}, total_sleep= 2.2, nnmf_dim=2, nnmf_error=2.2, time_id=2)>
<Frac(frac_id = 5, frac= test_frac4, best_params= {u'Age': 10, u'Name': u'Lara', u'Class': u'Fourth'}, best_val_score= 4.4, test_score=4.4, subject_id=3)>
<Frac(frac_id = 6, frac= test_frac5, best_params= {u'Age': 11, u'Name': u'Zara', u'Class': u'First'}, best_val_score= 5.5, test_score=5.5, subject_id=3)>
<Subject(subject_id = 4, subject= test_subject3, class_prior= {u'Age': 10, u'Name': u'Lara', u'Class': u'Fourth'}, total_sleep= 3.3, nnmf_dim=3, nnmf_error=3.3, time_id=2)>
<Frac(frac_id = 7, frac= test_frac5, best_params= {u'Age': 11, u'Name': u'Zara', u'Class': u'First'}, best_val_score= 5.5, test_score=5.5, subject_id=4)>
<Frac(frac_id = 8, frac= test_frac6, best_